In [10]:
import networkx as nx

model = nx.DiGraph([
            ("P", "N"),
            ("P", "C"),
            ("C", "N"),
        ])

[ee](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098140243/files/assets/ciip_03in11.png )

In [13]:
print(not(nx.d_separated(model, {"C"}, {"N"}, {'P'})))

True
